# Practical Time Series Analysis - CH02 Python Code Snippets

## Assembling a Time Series Data Collection

In [4]:
import pandas as pd

In [64]:
# Load year joined data
YearJoined = pd.read_csv('../data/year-joined.csv')
YearJoined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


In [11]:
# Check whether a member has more than one entry
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [65]:
# Load emails data
emails = pd.read_csv('../data/emails.csv')
emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


In [15]:
# Check whether email opening null weeks are reported
emails[emails['emailsOpened']<1]

,emailsOpened,user,week


In [59]:
# History of a selected user
emails[emails['user']==998].head()

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00


In [30]:
# How many week observations should we have between the first and last event for that member?
print(
    int((
    pd.to_datetime(max(emails[emails['user']==998]['week'])) - pd.to_datetime(min(emails[emails['user']==998]['week']))
    ).days/7)
)

# How many weeks of data do we have for that member?
print(emails[emails['user']==998].shape[0])

25
24


In [33]:
# Fill in missing weeks for all members using Pandas' indexing
complete_idx = pd.MultiIndex.from_product(
    (set(emails['week']), set(emails['user']))
)

all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']

In [60]:
# Look at user 998 again
all_email[all_email['user']==998].sort_values('week').head()

,week,user,emailsOpened
54977,2017-12-04 00:00:00,998.0,1.0
80310,2017-12-11 00:00:00,998.0,3.0
73842,2017-12-18 00:00:00,998.0,3.0
85161,2017-12-25 00:00:00,998.0,0.0
39885,2018-01-01 00:00:00,998.0,3.0


In [55]:
# For each user determine the start_date and end_date cutoffs
cutoff_dates = emails.groupby('user')['week'].agg(['min', 'max']).reset_index()

In [56]:
# Drop rows from the data that don't contribute to the chronology
for _, row in cutoff_dates.iterrows():
    user = row['user']
    start_date = row['min']
    end_date = row['max']
    
    all_email.drop(all_email[all_email['user']==user][all_email['week']<start_date].index, 
                   inplace=True)
    all_email.drop(all_email[all_email['user']==user][all_email['week']>end_date].index, 
                   inplace=True)

<ipython-input-56-623d738c1157>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email['user']==user][all_email['week']<start_date].index,
<ipython-input-56-623d738c1157>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email['user']==user][all_email['week']>end_date].index,


In [61]:
all_email.head()

,week,user,emailsOpened
0,2018-01-15 00:00:00,1.0,3.0
2,2018-01-15 00:00:00,5.0,0.0
3,2018-01-15 00:00:00,6.0,2.0
4,2018-01-15 00:00:00,9.0,3.0
6,2018-01-15 00:00:00,14.0,1.0


## Constructing a Found Time Series

In [77]:
# Load donations data
donations = pd.read_csv('../data/donations.csv')
donations.head()

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


In [78]:
# Resample donations data at a weekly frequency
donations['timestamp'] = pd.to_datetime(donations['timestamp'])
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(
    lambda df: df['amount'].resample('W-MON').sum().dropna()
).reset_index()

In [79]:
agg_don.head()

,user,timestamp,amount
0,0.0,2015-03-30,25.0
1,0.0,2015-04-06,0.0
2,0.0,2015-04-13,0.0
3,0.0,2015-04-20,0.0
4,0.0,2015-04-27,0.0


In [142]:
# Join donations data and email data
merged_df = pd.DataFrame([])
for user, user_email in all_email.groupby('user'):
    user_donations = agg_don[agg_don['user']==user]
    user_donations.set_index('timestamp', inplace=True)
    
    user_email = all_email[all_email['user']==user].sort_values('week')
    user_email['week'] = pd.to_datetime(user_email['week'])
    user_email.set_index('week', inplace=True)
    
    df = pd.merge(user_email, user_donations, 
                  how='left', left_index=True, right_index=True)
    df.fillna(0, inplace=True)
    df['user'] = df['user_x']
    
    merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])

In [157]:
# Shift donation a week forward
df = merged_df[merged_df['user']==998]
df['target'] = df['amount'].shift(1)
df = df.fillna(0)

<ipython-input-157-baa81b1c8657>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df['amount'].shift(1)


In [159]:
df

,user,week,emailsOpened,amount,target
0,998.0,2017-12-04,1.0,0.0,0.0
1,998.0,2017-12-11,3.0,0.0,0.0
2,998.0,2017-12-18,3.0,0.0,0.0
3,998.0,2017-12-25,0.0,0.0,0.0
4,998.0,2018-01-01,3.0,0.0,0.0
5,998.0,2018-01-08,3.0,50.0,0.0
6,998.0,2018-01-15,2.0,0.0,50.0
7,998.0,2018-01-22,3.0,0.0,0.0
8,998.0,2018-01-29,2.0,0.0,0.0
9,998.0,2018-02-05,3.0,0.0,0.0
